<a href="https://colab.research.google.com/github/nnilayy/CAFA-5-Protein-Function-Prediction/blob/main/00_mdna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Setting up

In [ ]:
!pip install sec-parser tiktoken jupyter_black sec_downloader

In [ ]:
from __future__ import annotations
import sec_parser as sp
import sec_downloader as sd
import jupyter_black
import tiktoken

jupyter_black.load()

In [ ]:
!git clone https://github.com/alphanome-ai/sec-parser
%cd "/content/sec-parser"

In [ ]:
!pip install poetry
!poetry export -f requirements.txt --output requirements.txt
!pip install -r "/content/sec-parser/requirements.txt"

## Ticker

In [1]:
ticker="SBUX"

## Ticker Data

In [4]:
# Download HTML
dl = sd.Downloader("alphanome.ai", "info@alphanome.ai")
html = dl.get_filing_html(form="10-Q",ticker=ticker)
elements = sp.Edgar10QParser().parse(html)
table_elements=[e for e in elements if isinstance(e,sp.TableElement)]
len(table_elements)

51

## HTML

In [ ]:
import re

pattern = r"\[Table with \d+ rows, \d+ numbers, and \d+ characters.]"
matches = re.findall(pattern, markdown)
for i, match in zip(range(len(matches)),matches):
    print(str(i)+" "+match)

In [ ]:
from bs4 import BeautifulSoup
index=8
html=table_elements[index].html_tag.get_source_code()
soup=BeautifulSoup(html,'html.parser')
soup

## OUTPUT

In [9]:
index=0

In [13]:
from sec_parser.semantic_elements.table_element.table_parser import TableParser
index=index+1
print(ticker+"-"+str(index)+"/"+str(len(table_elements)))
table=table_elements[index].html_tag.get_source_code()
df=TableParser(table).parse_as_df()
df

AAPL-4/34


,1,2,3,4,5
0,,Three Months Ended,Three Months Ended,Nine Months Ended,Nine Months Ended
1,,"July 1, 2023","June 25, 2022","July 1, 2023","June 25, 2022"
2,Net sales:,,,,
3,Products,$60584,$63355,$230901,$245241
4,Services,21213,19604,62886,58941
5,Total net sales,81797,82959,293787,304182
6,Cost of sales:,,,,
7,Products,39136,41485,146696,155084
8,Services,6248,5589,18370,16411
9,Total cost of sales,45384,47074,165066,171495


## Preprocessing Functions

In [126]:
import pandas as pd
from io import StringIO
import re

def basic_preprocessing(html: str) -> pd.DataFrame:
  tables = pd.read_html(StringIO(html), flavor="lxml")
  if len(tables) == 0:
      msg = "No tables found"
      raise ValueError(msg)
  table = tables[0]
  table = table.dropna(how="all")
  table.columns = pd.Index(table.iloc[0].tolist())
  table = table[1:]
  table = table.dropna(axis=1, how="all")
  first_row = list(table.columns)
  new_df = pd.DataFrame([first_row], columns=table.columns)
  table = pd.concat([new_df, table], ignore_index=True)
  table.columns = pd.Index(
      [i for i in range(1, len(table.columns) + 1)],
      )
  table = table.fillna("")
  table=table.astype(str)
  table = table.applymap(lambda x: x[:-2] if isinstance(x, str) and x.endswith('.0') else x)
  return table


def remove_blank_columns(df: pd.DataFrame) -> pd.DataFrame:
    columns_to_remove = []
    for i, _ in enumerate(df.columns):
        if i < len(df.columns) - 1:
            current_column = df.columns[i]
            next_column = df.columns[i + 1]
            non_empty_rows = df[
                (df[current_column] != "") & (df[next_column] != "")
            ]
            if (
                non_empty_rows[current_column] == non_empty_rows[next_column]
            ).all():
                if (df[current_column] == "").sum() >= (
                    df[next_column] == ""
                ).sum():
                    columns_to_remove.append(current_column)
                else:
                    columns_to_remove.append(next_column)
    return df.drop(columns=columns_to_remove)


def merge_columns_by_markers(table: pd.DataFrame, markers: list) -> pd.DataFrame:
    left_merging_markers=['%', ')', 'pts', 'pts.', '*', '**',"%)",""]
    right_merging_markers=["$"]

    for marker in markers:
        marker_columns = [col for col in table.columns if any(str(value) == marker for value in table[col])]
        for current_column in marker_columns:
            if current_column not in table.columns:
              continue

            current_column_index = table.columns.get_loc(current_column)

            if marker in right_merging_markers and current_column_index < len(table.columns) - 1:
                left_column=current_column
                right_column = table.columns[current_column_index + 1]

            elif marker in left_merging_markers and current_column_index > 0:
                left_column = table.columns[current_column_index - 1]
                right_column=current_column
            else:
                continue

            non_marker_rows = table[(~table[left_column].isin(left_merging_markers+right_merging_markers)) &
                                    (~table[right_column].isin(left_merging_markers+right_merging_markers))]
            non_empty_non_marker_rows = non_marker_rows[(non_marker_rows[left_column] != "") & (non_marker_rows[right_column] != "")]
            marker_rows = table.drop(non_empty_non_marker_rows.index)
            marker_rows_indices = marker_rows.index

            if (non_empty_non_marker_rows[left_column] == non_empty_non_marker_rows[right_column]).all():
                table.loc[marker_rows_indices, left_column] = (table.loc[marker_rows_indices, left_column].str.cat(table.loc[marker_rows_indices, right_column], sep="", na_rep="").str.strip())
                table = table.drop(columns=[right_column])
    return table


# -------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------------------

def assign_same_value(df):
    first_column = df.columns[0]
    for index, row in df.iterrows():
        if row[first_column] == '' and all(value == row.iloc[1] for value in row.iloc[1:]):
            value_to_assign = row.iloc[1]
            df.loc[index, first_column] = value_to_assign
    return df


def clean_repeating_rows(df):
  same_values_mask = df.apply(lambda row: all(value == '' or value == row.iloc[0] for value in row), axis=1)
  df.loc[same_values_mask, df.columns[1:]] = ''
  return df


def row_splitter(df):
    df['Marked'] = (df.apply(lambda row: any(bool(re.search(r'\$\s*\(\s*[\d,]+\s*\)|\$\s*[\d,]+|\(\s*[\d,]+\s*%\s*\)|[\d,]+\s*%',
                                                            str(cell).replace(" ", ""))) for cell in row), axis=1))
    first_true_index = df.index[df['Marked']].min()
    df=df.drop(columns=["Marked"])
    header_rows = df.iloc[:first_true_index, :]
    df_remaining = df.iloc[first_true_index:, :]
    return header_rows, df_remaining


def redundant_adjacent_row_cleaner(column):
    for i in range(1, len(column)):
        if column[i - 1] == column[i]:
            column[i - 1] = ''
    return column

# -------------------------------------------------------------------------------------------------------------------------
# -------------------------------------------------------------------------------------------------------------------------
def column_to_index(df):
  first_column = df.columns[0]
  column_values = df[first_column].values
  df=df.set_index(column_values)
  df=df.drop(columns=[first_column])
  return df


def multi_indexer(header_rows,df_remaining):
  header_rows=column_to_index(header_rows)
  df_remaining=column_to_index(df_remaining)

  num_rows=len(header_rows)
  rows_list = [header_rows.iloc[i].tolist() for i in range(num_rows)]
  tupled_indices = list(zip(*rows_list))
  # level_names = [f"lvl-{i}" for i in range(num_rows)]
  multi_index_columns=pd.MultiIndex.from_tuples(tupled_indices, names=header_rows.index)
  df_remaining.columns=multi_index_columns
  # df_remaining=df_remaining.reset_index(drop=True)
  return df_remaining

## Output

In [397]:
index=0

In [481]:
index=index+1
print(ticker+"-"+str(index)+"/"+str(len(table_elements)))
table=table_elements[index].html_tag.get_source_code()
based_df=basic_preprocessing(table)
based_df=remove_blank_columns(based_df)
based_df=merge_columns_by_markers(based_df,["","$","%",")","%)","pts","pts.","*","**"])
based_df.columns = range(1, len(based_df.columns) + 1)
based_df

SBUX-30/51


,1,2,3,4,5
0,,Quarter Ended,Quarter Ended,Three Quarters Ended,Three Quarters Ended
1,,"Jul 2, 2023","Jul 3, 2022","Jul 2, 2023","Jul 3, 2022"
2,Operating lease costs(1),$401.6,$386.5,$1188.2,$1166
3,Variable lease costs,264.1,221.5,753.3,687.1
4,Short-term lease costs,7,6.9,20.9,21.1
5,Total lease costs,$672.7,$614.9,$1962.4,$1874.2


In [482]:
# Splitting df into header and remaining rows
header_rows,df_remaining=row_splitter(based_df)

# Cleaning Header Rows
header_rows=header_rows.apply(redundant_adjacent_row_cleaner, axis=0)

# Cleaning Remaining Rows
df_remaining=assign_same_value(df_remaining)
df_remaining=clean_repeating_rows(df_remaining)
data=multi_indexer(header_rows,df_remaining)
# data

In [483]:
header_rows

,1,2,3,4,5
0,,Quarter Ended,Quarter Ended,Three Quarters Ended,Three Quarters Ended
1,,"Jul 2, 2023","Jul 3, 2022","Jul 2, 2023","Jul 3, 2022"


## Standard Table Function

In [458]:
import re
import numpy as np
def extracting_subsets(df):
  df=df.iloc[:, 1:]
  mask1 = df.apply(lambda row: all(cell == '' for cell in row), axis=1)

  pattern = r'\$\s*\(\s*[\d,]+\s*\)|\$\s*[\d,]+|\(\s*[\d,]+\s*%\s*\)|[\d,]+\s*%'
  mask2 = df.apply(lambda x: pd.notna(re.search(pattern, str(x))),axis=1)

  df_filtered = df[~mask1 & ~mask2]
  duplicated_rows = df_filtered[df_filtered.duplicated(keep=False)]

  matching_indices = list(duplicated_rows.index)
  return matching_indices


def date_rows(df):
    # Finding starting row
    pattern = re.compile(r'(?:months?|quarters?) ended|quarter', re.IGNORECASE)
    starting_rows = df[df.applymap(lambda cell: bool(re.search(pattern, str(cell)))).any(axis=1)]
    starting_indices = starting_rows.index.tolist()
    starting_index = starting_indices[0] if starting_indices else None

    # Finding ending row
    ending_rows = df[df.applymap(lambda cell: bool(re.search(r'\b[12]\d{3}\b', str(cell)))).any(axis=1)]
    ending_indices = ending_rows.index.tolist()
    ending_index = ending_indices[-1] if ending_indices else None

    if starting_index and ending_index is None:
      ending_index=starting_index
    elif ending_index and starting_index is None:
      starting_index=ending_index
    elif starting_index is None and ending_index is None:
      starting_index=ending_index=None

    # Extracting rows from starting_index to ending_index
    rows = df.loc[starting_index:ending_index]
    return rows



def date_pattern_finder(df):
    names=['Month-Ended', 'Month', 'Year']
    patterns = [
        r'(?:months?|quarters?) ended|quarter',
        r'\d{1,2}\s+(?:January|February|March|April|May|June|July|August|September|October|November|December)|(?:January|February|March|April|May|June|July|August|September|October|November|December)\s+\d{1,2}',
        r'\b[12]\d{3}\b'
    ]

    result_dict = {}
    for name, pattern in zip(names, patterns):
      pattern = re.compile(pattern, re.IGNORECASE)
      result_dict[name] = {column: df[df[column].apply(lambda cell: bool(re.search(pattern, str(cell))))].index.tolist() for column in df.columns}
    return result_dict


def date_row_unifier(df):
  pattern_df=pd.DataFrame(date_pattern_finder(df)).T
  pattern_df=pattern_df.applymap(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
  pattern_df=pattern_df.replace({np.nan: None}, inplace=True)

  for column in df.columns:
    month_end_value=pattern_df.loc["Month-Ended", column]
    month_value=pattern_df.loc["Month", column]
    year_value=pattern_df.loc["Year", column]

    if month_end_value==None and month_value==None and year_value==None:
      continue
    elif month_end_value==None and month_value==year_value:
      continue
    else:
      if year_value!=month_value:
        df.loc[month_value, column] =df.loc[month_value, column] +", "+ df.loc[year_value, column]
        df.loc[year_value, column] = ""

      elif year_value!=month_end_value:
        df.loc[month_end_value, column]=df.loc[month_end_value, column] +", "+ df.loc[year_value, column]
        df.loc[year_value, column] = ""

  return df

## Output

In [484]:
df=date_rows(header_rows).copy()
df

,1,2,3,4,5
0,,Quarter Ended,Quarter Ended,Three Quarters Ended,Three Quarters Ended
1,,"Jul 2, 2023","Jul 3, 2022","Jul 2, 2023","Jul 3, 2022"


In [485]:
pattern_df=pd.DataFrame(date_pattern_finder(df)).T
# pattern_df=pattern_df.applymap(lambda x: x[0] if isinstance(x, list) and len(x) > 0 else None)
# pattern_df=pattern_df.replace({np.nan: None})
pattern_df

,1,2,3,4,5
Month-Ended,[],[0],[0],[0],[0]
Month,[],[],[],[],[]
Year,[],[1],[1],[1],[1]


In [ ]:
date_unified_header_rows=date_row_unifier(df)
header_rows.update(date_unified_header_rows)
date_unified_table=pd.concat([header_rows,df_remaining])
date_unified_table

## Future

In [ ]:
# def contains_subtables(df):
# def is_numeric_table(df):
# def correct_index(df):
# cvx-4 works for no index names, something else is the issue

In [ ]:
import pandas as pd

# Sample DataFrame
data = {'A': [1, 2, 2, 3, 3],
        'B': [1, 5, 5, 6, 6],
        'C': [1, 8, 8, 9, 9]}

df = pd.DataFrame(data)

# Function to create a multi-index based on consecutive duplicates
def create_multi_index(row):
    groups = (row != row.shift()).cumsum()
    return pd.MultiIndex.from_arrays([row, groups], names=['Value', 'Group'])

# Apply the function to each row and transpose the result
multi_index = df.apply(create_multi_index, axis=1).transpose()

# Set the multi-index to the DataFrame columns
df.columns = multi_index

# Define a custom style function to hide redundant data
def hide_redundant_data(val):
    return 'color: white' if pd.notna(val) and len(set(val)) == 1 else ''

# Apply the style to the DataFrame
styled_df = df.style.applymap(hide_redundant_data)

# Display the styled DataFrame
styled_df



ValueError: ignored

In [ ]:
df.style.set_table_styles([{'selector': 'level0', 'props': [('display', 'none')]}, {'selector': 'thead th', 'props': [('display', 'none')]}])

,A,B,C
"MultiIndex([(1, 1), (1, 1), (1, 1)], names=['Value', 'Group'])",1,1,1
"MultiIndex([(2, 1), (5, 2), (8, 3)], names=['Value', 'Group'])",2,5,8
"MultiIndex([(2, 1), (5, 2), (8, 3)], names=['Value', 'Group'])",2,5,8
"MultiIndex([(3, 1), (6, 2), (9, 3)], names=['Value', 'Group'])",3,6,9
"MultiIndex([(3, 1), (6, 2), (9, 3)], names=['Value', 'Group'])",3,6,9
